In [ ]:
#0) What: Test steps 1,2,3 in local Postgres instance
#Who: Bryan/Jesse

#1) What: Create a second table(s) within Postgres. I suggest just adding "_cleaned" to the name and use the same prefix. 
#The schema will remain the same. William - any objections here? We don't have a dev/staging Postgres instance, do we? 
#Who: Jesse

#2a) What: Take Bryan's new Python and test each 'rule,' adding logic for writing to Postgres/the new 'cleaned' table.
#Who: Bryan/Jesse

#2b) What: Add logic for alerts (SendGrid); fire alert when issue found, correction made.
#Who: Jesse

#3) What: Verify that results look good in *_cleaned table(s).
#Who: Bryan/Jesse/William(?)/anyone else who wants to help test

#4) What: Set up cron job to run, initially, every 4-6 hours. Review results and make sure that:
#Data is not thrown out, unnecessarily altered/integrity is retained
#Any data corrections are done properly
#Review / tweak anything else (see Bryan's notes in Asana: "Data Cleaning Notes" under "Data Science Brainstorming")
#Who: Bryan/Jesse

#5) What: After 5-7 days of testing, 'deploy.' Cron job does not need to run as often; decide on interval. 
#Who: Jesse

#6) What: Point all relevant queries to *_cleaned table(s)
#Who: Bryan/Jesse/Emily


#Will it tell the difference between an account that is consistently down or inconstently down.
#After missing a reading, does a sensor ever get back to 100% (visualize this)
#csv of sensor ids, date, and percentage (goes to a website?)
#can it pull the peron's first and last names 
#first name last name (sensor id), dates, percentages below

#Fix tennant names


#Code to visualize stuff - for time periods for month and years

In [1]:
import datetime
import pandas as pd
import numpy as np
import psycopg2
import csv
import time
import datetime

In [127]:
try:
    connection = psycopg2.connect(database ='heatseek', user = 'heatseekroot', password = 'wearecoolsoweseekheat')
    cursor = connection.cursor() #Open a cursor to perform operations
    
    cursor.execute('SELECT * from users') #Executes the query
    users = cursor.fetchall() #cursor.fetchone() for one line, fetchmany() for multiple lines, fetchall() for all lines
    users = pd.DataFrame(users) #Saves 'users' as a pandas dataframe
    users_header = [desc[0] for desc in cursor.description] #This gets the descriptions from cursor.description 
    #(names are in the 0th index)
    users.columns = users_header #PD array's column names
    
    cursor.execute('SELECT * FROM readings;')
    readings = cursor.fetchall()
    readings = pd.DataFrame(readings)
    readings_header = [desc[0] for desc in cursor.description]
    readings.columns = readings_header
    
    cursor.execute('SELECT * FROM sensors;')
    sensors = cursor.fetchall()
    sensors = pd.DataFrame(sensors)
    sensors_header = [desc[0] for desc in cursor.description]
    sensors.columns = sensors_header
    
    cursor.close() 
    connection.close()
    
except psycopg2.DatabaseError, error:
    print 'Error %s' % error

In [159]:
#This creates an array 'sensors_with_users' that consists of sensors that are currently assigned to users.
sensors_with_users_raw = np.intersect1d(users.id.unique(), sensors.user_id.unique()) #Returns the common ids in both the datasets.
#sensors.loc[sensors.user_id, sensors_with_users]
sensors_with_users = []
for ids in sensors_with_users_raw:
    sensors_with_users.append(int(ids))

In [164]:
#This function returns clean readings. #It doesn't exist yet
#This function will return if a sensor is polling faster than once per hour (i.e., test cases)

def dirty_data(dirty_readings, start_date = None, end_date = None):
    if (start_date or end_date) == None:
        start_date = pd.Timestamp('2000-01-01')
        end_date = pd.Timestamp(datetime.datetime.now())
    else:
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)
    
    mask = (dirty_readings['created_at'] > start_date) & (dirty_readings['created_at'] <= end_date)
    dirty_readings = dirty_readings.loc[mask]
    
    hot_ids = dirty_readings.loc[dirty_readings.temp > 90].sensor_id.unique() #Returns sensor IDs where indoor temp is > 90
    cold_ids = dirty_readings.loc[dirty_readings.temp < 40].sensor_id.unique() #Returns sensor IDs where indoor temp is < 40
    inside_colder_ids = dirty_readings.loc[dirty_readings.temp < dirty_readings.outdoor_temp].sensor_id.unique() #Returns sensor IDs where indoor temp is < outdoor temp
    #Array of all the IDs above
    all_ids = np.unique(np.concatenate((hot_ids, cold_ids, inside_colder_ids)))
    all_ids = all_ids[~np.isnan(all_ids)]
    #Create an empty dataframe with the IDs as indices
    report = pd.DataFrame(index=all_ids,columns=['UserID','SensorID', 'Outside90', 'Inside40', 'InsideColderOutside'])
    #Fill in the specific conditions as '1'
    report.Outside90 = report.loc[hot_ids].Outside90.fillna(1)
    report.Inside40 = report.loc[cold_ids].Inside40.fillna(1)
    report.InsideColderOutside = report.loc[inside_colder_ids].InsideColderOutside.fillna(1)
    report = report.fillna(0)
    report.SensorID = report.index
    
    #Fill in UserIDs
    problem_ids = sensors[sensors.id.isin(all_ids)]
    for index in report.index.values:
        index = int(index)
        try:
            report.loc[index, 'UserID'] = sensors.loc[index, 'user_id']
        except KeyError:
            report.loc[index, 'UserID']  = 'No such user in sensors table.'
    return report
        


def clean_data(dirty_readings):
    cleaner_readings = dirty_readings[dirty_readings.sensor_id.notnull()] #Remove cases where there are no sensor IDs
    return cleaner_readings
    
clean_readings = clean_data(readings)
report = dirty_data(readings)

In [6]:
#This function takes (start date, end date, sensor id), returns % of failure
def sensor_down(data, start_date, end_date, sensor_id): 
    
    #This pulls up the tennant's first and last name.
    try:
        tennant_id = int(sensors.loc[sensors.id == sensor_id].user_id.values[0])
        tennant_first_name = users.loc[users.id == tennant_id].first_name.values[-1] #This pulls up the first name on the list (not the most recent)
        tennant_last_name = users.loc[users.id == tennant_id].last_name.values[-1]
    #Are these really not assigned?
    except ValueError:
        tennant_id = 'None'
        tennant_first_name = 'Not'
        tennant_last_name = 'Assigned'
    except IndexError:
        tennant_id = 'None'
        tennant_first_name = 'Not'
        tennant_last_name = 'Assigned'
        
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)

    sensor_readings = data.loc[data.sensor_id == sensor_id]
    
    #Converting to timestamps
    #for i in sensor_readings.index.values: #Iterates through all the index values
        #sensor_readings.loc[i, 'created_at'] = pd.Timestamp(sensor_readings.created_at[i])
    #Using map instead of for loop (about 15-20x faster)
    try:
        sensor_readings.loc[:, 'created_at'] = map(pd.Timestamp, sensor_readings.created_at)
    except TypeError:
        tennant_first_name = 'Mapping Error'
        tennant_last_name = 'Only One Entry'
        pass
    #Using list comprehensions (as efficient as map)
    #sensor_readings.loc[:, 'created_at'] = [pd.Timestamp(x) for x in sensor_readings.created_at]
        
    #Using a boolean mask to select readings between the two dates 
    #(http://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates)   
    mask = (sensor_readings['created_at'] > start_date) & (sensor_readings['created_at'] <= end_date)
    masked_sensor_readings = sensor_readings.loc[mask] #Get all readings between the two dates
    masked_sensor_readings = masked_sensor_readings.sort_values('created_at')
    #We then calculate how many hours have passed for that specific sensor and date range
    try:
        sensor_readings_start_date = masked_sensor_readings.loc[masked_sensor_readings.index.values[0], 'created_at']
        sensor_readings_end_date = \
        masked_sensor_readings.loc[masked_sensor_readings.index.values[len(masked_sensor_readings)-1], 'created_at']
        timedelta_in_seconds =  sensor_readings_end_date - sensor_readings_start_date #This returns Timedelta object
        timedelta_in_seconds = timedelta_in_seconds.total_seconds()
        total_number_of_hours = timedelta_in_seconds/3600 + 1 #The +1 fixes the rounding error for now but IDK why yet.
        
        hours_in_date_range = ((end_date-start_date).total_seconds())/3600 + 1
        
    except IndexError:
        return [tennant_first_name, tennant_last_name, sensor_id, tennant_id, "No valid readings during this time frame."]
    
    proportion_of_total_uptime = (len(masked_sensor_readings)/hours_in_date_range) * 100 #Proportion of uptime over TOTAL HOURS
    proportion_within_sensor_uptime = (len(masked_sensor_readings)/total_number_of_hours) * 100 #Proportion of uptime for the sensor's first and last uploaded dates.
    if proportion_within_sensor_uptime <= 100.1:
        return [tennant_first_name, tennant_last_name, sensor_id, tennant_id, proportion_of_total_uptime, proportion_within_sensor_uptime]
    else:
        return [tennant_first_name, tennant_last_name, sensor_id, tennant_id, proportion_of_total_uptime, proportion_within_sensor_uptime, 'Sensor has readings more frequent than once per hour. Check readings table.']

In [7]:
print sensor_down(readings, '2016-02-01', '2016-02-07', 19)
#mask = (readings['created_at'] > pd.Timestamp('2016-02-01')) & (readings['created_at'] <= pd.Timestamp('2016-02-07'))
#readings.loc[readings.sensor_id == 296]

['Loraine', 'Dellamore', 19, 245, 99.31034482758618, 99.99980709913753]


In [8]:
#This function creates a simulated dataset of readings.
def simulate_data(start_date, end_date, polling_rate, sensor_id): #polling_rate in minutes
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)

    #how many hours between the two dates:
    timedelta_in_seconds = end_date-start_date
    total_number_of_hours = timedelta_in_seconds.total_seconds()/(polling_rate*60)
    
    #Create an empty pandas dataframe
    index = xrange(1,int(total_number_of_hours)+1)
    columns = ['created_at', 'sensor_id']
    simulated_readings = pd.DataFrame(index = index, columns = columns)
    simulated_readings.loc[:,'sensor_id'] = sensor_id
    
    #Populate it with columns of 'create_at' dates
    time_counter = start_date
    for i in simulated_readings.index.values:
        simulated_readings.loc[i,'created_at'] = time_counter
        time_counter = time_counter + pd.Timedelta('00:%s:00' % polling_rate)
   
    return simulated_readings

In [65]:
#Testing stuff
#simulated_data = simulate_data('2015-01-01', '2015-01-31', 60, 30)
#print sensor_down(readings, '2015-01-01', '2016-01-01', 26) #I think this works
#print sensor_down(simulated_data, '2015-01-01', '2015-01-31', 30) #There's some rounding error
#for user_id in sensors_with_users:
    #latest_id = sensors.loc[sensors.user_id==ids].id
    #latest_id = latest_id.index.values[len(latest_id)-1]
    #print sensors.loc[sensors.user_id == ids].id
    #try:
    #print sensor_down(readings, '2015-01-01', '2016-01-01', sensors.loc[sensors.user_id == user_id].id.values[0])
    #print users.loc[users.id == user_id].first_name
    #except IndexError:
    #print "Error."
    #print sensors.loc[sensors.user_id == user_id].id.values[0]

In [9]:
#This function generates a report; we might want to make this a cron job.
def generate_report(start_date, end_date):
    report = []
    sensor_ids = readings.sensor_id.unique()
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    for ids in sensor_ids:
        report.append(sensor_down(readings, start_date, end_date, ids))
    return report

In [10]:
tic = time.clock()

report = generate_report('2016-02-01','2016-02-07')
header =['FirstName', 'LastName', 'SensorID', 'UserID', 'Percentage of uptime in daterange', 'Percentage of uptime while sensor was transmitting']

toc = time.clock()
toc - tic

3.59626564107334

In [79]:
csvoutput = open('report.csv', 'wb')
writer = csv.writer(csvoutput)
writer.writerow(header)
for i in report:
    writer.writerow(i)
csvoutput.close()

In [98]:
#What criteria do we want to use.

#SENSOR NUMBERS AS IDENTIFIERS, SOME DEGREE OF SEVERITY OF PROBLEM (CATEGORICAL)

#for the criteria we go, how do we define "bad"
#total days
#proportion of days (or clusters?)
#temperature discrepancy
#multiple apartments in the same building (how many % of the apartments in a building and how bad)
#multiple buildings by the same landlord (how many % of the buildings a landlord owns are bad)
#are our sensors failing in a specific building?

#Getting rid of test cases:
#1. Can we just delete by test IDs?
#2. If testing was separated by a minute, we can find all test cases by looping through all users,
#  and if they have a bunch of data that was collected within minutes, delete the user?

#We first convert the string dates into datetime format
sensordata['formatteddate'] = sensordata.created_at.apply(lambda x: pd.to_datetime(x,  format = "%Y-%m-%d %H:%M"))

#Then, one way of telling if a user_id was an actual user or a test case was to calculate the average timedelta for each user_id.
#Timedeltas of 1min are tests, 1 hour are users (don't know if this is always true, but if no user has an average polling rate
#of 1 min, we can use a bunch of methods to filter away test cases step by step).

sensordata['averagetimedelta'] = 0.00 #makes a new column

for i in sensordata.user_id.unique(): #for each user
    timelist = sensordata.loc[sensordata.user_id == i, 'formatteddate'] #this gives us a list of all their times in timestamp
    timedeltas = [] 
    for j in range(1, len(timelist)-1):
        timedeltas.append(timelist.iloc[j] - timelist.iloc[j-1]) #list of differences in time between time point j and j-1
    try:
        #we print the user_id, followd by their average time delta from point j to j-1
        print i, abs(sum(timedeltas, datetime.timedelta(0)))/len(timedeltas) #the average timedelta
    except ZeroDivisionError: #some cases have too few points (and results in a zero division error)
        #instead of breaking with we encounter a zerodivisionerror, just print the following:
        print i, "Too few data points?"
    sensordata.loc[sensordata.user_id == i, 'averagetimedelta'] = averagetimedeltas.total_seconds()

#3. If user ids are recycled, we'll have to do a combination of those things.

#Some sensors

NameError: name 'sensordata' is not defined

In [322]:
#Triage
#We want to have a measure of which users are facing the most chronic problems.

#Metric combining temperature difference and chronicity of problems

#Write some code that subsets all the violation == 't' cases
sensordataviolations = sensordata[sensordata.violation == 't'] #here it is.

#Hackiest method: just number of violations/numberof nonviolations and sort users by that
#That is, which users have had the most violations given the total number of readings 

violationsovertime = []

for i in sensordata.user_id.unique():
    nonviolations = sensordata.loc[sensordata.user_id == i, 'violation'].value_counts()['f'] #Number of violations = 'f'
    try:
        violations = sensordata.loc[sensordata.user_id == i, 'violation'].value_counts()['t'] #Number of violations = 't'
    except KeyError:
        violations = 0    
    sensordata.loc[sensordata.user_id == i, 'vfreq'] = float(violations)/float(nonviolations)
    violationsovertime.append([i, (float(violations)/float(nonviolations))])

#violations over time gives first the user_id, then the proportion of how many of their readings are violations

In [ ]:
#Stuff to do for fun

#Variability/consistency
#Which buildings have the least/most variable temperatures?
#For this, we just calculate within-person variability (how much do sensor temperatures by the same user) vary as a function of time
#We an use this same process to calculate variability between locations (e.g., just calculate variance for each location)

In [ ]:
#Now, we loop over all unique users in the dataset and generate a measure of how long they've had the sensor running
sensordata['totaltime'] = 0
sensordata['vfreq'] = 0

for i in sensordata.user_id.unique():
    firstentry = len(sensordata.loc[sensordata.user_id==i,'formatteddate']) #This gives us the index of the first timepoint
    lasttime = sensordata.loc[sensordata.user_id == i, 'formatteddate'].iloc[0] #This was the timestamp of the latest timepoint
    firsttime =  sensordata.loc[sensordata.user_id == i, 'formatteddate'].iloc[firstentry-1] #This was the timestamp of the first timepoint
    sensordata.loc[sensordata.user_id == i, 'totaltime'] = lasttime - firsttime #This is the timedelta (over how long a period readings were made)
    #print i, lasttime-firsttime

In [65]:
#sensors.user_id.value_counts()
#sensordata.violation.value_counts() #This returns the number of 't's and 'f's
#np.sort(userdata.id.unique())
#np.intersect1d(userdata.id.unique(), sensordata.user_id.unique()) #Returns the common ids in both the datasets.
#readings.sensor_id.unique

users.loc[users.last_name == "Quashie"]
#users.loc[users.id==160]
readings.loc[readings.user_id == 135].sort_values('created_at')
#users.loc[users.id == 236]

id  temp  twine_id          created_at                 updated_at  \
133078  526522    63       NaN 2015-02-19 06:09:09 2016-01-25 04:48:44.443183   
133075  526194    63       NaN 2015-02-19 21:09:09 2016-01-25 04:48:44.400827   
133081  526507    63       NaN 2015-02-19 22:09:09 2015-02-22 03:36:22.991260   
133080  526489    63       NaN 2015-02-19 23:09:09 2015-02-22 03:36:12.455004   
133077  526516    63       NaN 2015-02-20 00:09:09 2016-01-25 04:48:44.420321   
133076  526500    65       NaN 2015-02-20 01:09:09 2016-01-25 04:48:44.410440   
133083  526561    63       NaN 2015-02-22 04:09:09 2016-01-25 04:48:44.477740   
133086  526567    63       NaN 2015-02-22 05:09:09 2016-01-25 04:49:00.417950   
133097  526583    63       NaN 2015-02-22 06:09:10 2016-01-25 04:49:00.472798   
133098  526594    62       NaN 2015-02-22 07:09:09 2016-01-25 04:49:00.496304   
133101  526607    62       NaN 2015-02-22 08:09:10 2016-01-25 04:49:00.664260   
133102  526621    62       NaN 2015-02-22 09:09:09 2016-01-25 04:49:00.676312   
133104  526633    63       NaN 2015-02-22 10:09:09 2016-01-25 04:49:00.698500   
133108  526647    63       NaN 2015-02-22 11:09:10 2016-01-25 04:49:00.743618   
133110  526657    65       NaN 2015-02-22 12:09:09 2016-01-25 04:49:00.765060   
133148  526668    65       NaN 2015-02-22 13:09:10 2016-01-25 04:49:01.226855   
133151  526680    65       NaN 2015-02-22 14:09:09 2016-01-25 04:49:01.276720   
133115  526694    65       NaN 2015-02-22 15:09:09 2016-01-25 04:49:00.832766   
133120  526704    65       NaN 2015-02-22 16:09:10 2016-01-25 04:49:00.885975   
133123  526713    65       NaN 2015-02-22 17:09:10 2016-01-25 04:49:00.919456   
133127  526724    66       NaN 2015-02-22 18:09:10 2016-01-25 04:49:00.969386   
133131  526737    66       NaN 2015-02-22 19:09:10 2016-01-25 04:49:01.057189   
133154  526744    66       NaN 2015-02-22 20:09:10 2016-01-25 04:49:01.312667   
133160  526760    65       NaN 2015-02-22 21:09:10 2016-01-25 04:49:01.382816   
133165  526772    65       NaN 2015-02-22 22:09:10 2016-01-25 04:49:01.443789   
133169  526783    66       NaN 2015-02-22 23:09:10 2016-01-25 04:49:01.490072   
643     526791    66       NaN 2015-02-23 00:09:10 2016-01-25 04:54:29.637176   
758     526801    66       NaN 2015-02-23 01:09:10 2016-01-25 04:54:29.692262   
915     526814    65       NaN 2015-02-23 02:09:10 2016-01-25 04:54:46.302491   
918     526820    66       NaN 2015-02-23 03:09:10 2016-01-25 04:54:46.337705   
...        ...   ...       ...                 ...                        ...   
125419  594767   112       NaN 2016-02-13 15:03:10 2016-02-13 18:18:12.300879   
125449  594797   111       NaN 2016-02-13 16:03:10 2016-02-13 18:18:53.687502   
125574  594827   110       NaN 2016-02-13 17:03:10 2016-02-13 18:19:35.305091   
125602  594855   112       NaN 2016-02-13 18:03:10 2016-02-13 18:24:07.053451   
125923  594886   111       NaN 2016-02-13 19:03:10 2016-02-13 19:08:57.442103   
125727  594916   110       NaN 2016-02-13 20:03:10 2016-02-13 20:09:46.917118   
125756  594945   112       NaN 2016-02-13 21:03:10 2016-02-13 21:10:34.835127   
125786  594975   111       NaN 2016-02-13 22:03:10 2016-02-13 22:07:27.662220   
125939  595003   112       NaN 2016-02-13 23:03:10 2016-02-13 23:08:14.294838   
126303  595200   112       NaN 2016-02-14 00:03:10 2016-02-14 00:07:53.944439   
126332  595229   112       NaN 2016-02-14 01:03:10 2016-02-14 01:08:41.442709   
126390  595259   112       NaN 2016-02-14 02:03:10 2016-02-14 02:09:32.620839   
126443  595288   113       NaN 2016-02-14 03:03:10 2016-02-15 16:33:29.935914   
126501  595317   112       NaN 2016-02-14 04:03:10 2016-02-15 16:34:00.968598   
126544  595345   111       NaN 2016-02-14 05:03:10 2016-02-15 16:34:31.048499   
126570  595371   110       NaN 2016-02-14 06:03:10 2016-02-15 16:34:59.017811   
126599  595400   112       NaN 2016-02-14 07:03:10 2016-02-15 16:35:30.130145   
126733  595429   113       NaN 2016-02-14 0